In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

Proof that reducing r^2 gives a hebbian learning rule. also, Es are reduced when weights are random. For Scalars

In [24]:
U1 = nn.Linear(1,1, bias=False)
# U2 = nn.Linear(1,1, bias=False)

# def step(x, e1, r1, e2, r2):
def step(x, e1, r1):
    with torch.no_grad():
        e1 = x - torch.matmul(U1.weight.T, r1)
        # e2 = r1 - torch.matmul(U2.weight.T, r2)
    r1 = r1 + U1(e1) 
    # r2 = r2 + U2(e2)
    # return e1, r1, e2, r2
    return e1, r1

In [100]:
x = torch.rand(1)
e1 = torch.rand(1)
r1 = torch.rand(1)
# e2 = torch.rand(1)
# r2 = torch.rand(1)

# print all
print(f"x: {x}")
print(f"e1: {e1}")
print(f"r1: {r1}")
# print(f"e2: {e2}")
# print(f"r2: {r2}")

x: tensor([0.8994])
e1: tensor([0.4245])
r1: tensor([0.5636])


In [111]:
e1 = e1.detach()
r1 = r1.detach()
# e2 = e2.detach()
# r2 = r2.detach()
# e1, r1, e2, r2 = step(x, e1, r1, e2, r2)
e1, r1 = step(x, e1, r1)
print(f" x: {x}")
print(f"e1: {e1}")
print(f"r1: {r1}")
print(f"Ur1: {torch.matmul(U1.weight.T, r1)}")
# print(f"e2: {e2}")
# print(f"r2: {r2}")

 x: tensor([0.8994])
e1: tensor([0.0204])
r1: tensor([1.6528], grad_fn=<AddBackward0>)
Ur1: tensor([0.8848], grad_fn=<MvBackward0>)


In [15]:
with torch.no_grad():
    U1.weight.grad = torch.zeros_like(U1.weight)
    U2.weight.grad = torch.zeros_like(U2.weight)
loss = 0.5*((r1**2).sum() + (r2**2).sum())
loss.sum().backward()
print(f'U1.weight.grad:\n{U1.weight.grad}')
print(f'U2.weight.grad:\n{U2.weight.grad}')

U1.weight.grad:
tensor([[0.0049]])
U2.weight.grad:
tensor([[0.0075]])


In [17]:
print(f'1:\n{torch.matmul(e1.unsqueeze(1), r1.unsqueeze(1).T).T}')
print(f'2:\n{torch.matmul(e2.unsqueeze(1), r2.unsqueeze(1).T).T}')

1:
tensor([[0.0049]], grad_fn=<PermuteBackward0>)
2:
tensor([[0.0075]], grad_fn=<PermuteBackward0>)


Expansion to Vector Es and Rs

In [310]:
torch.manual_seed(42)
U1 = nn.Linear(2,4, bias=False)
U2 = nn.Linear(4,6, bias=False)

# def step(x, e1, r1, e2, r2):
#     with torch.no_grad():
#         e1 = x - torch.matmul(U1.weight.T, r1)
#     r1 = r1 + U1(e1) 
#     with torch.no_grad():
#         # r1 -= (0.01*e2)
#         e2 = r1 - torch.matmul(U2.weight.T, r2)
#     r2 = r2 + U2(e2)
#     return e1, r1, e2, r2
def step(x, e1, r1, e2, r2):
    with torch.no_grad():
        e1 = x - torch.matmul(U1.weight.T, r1)
        e2 = r1 - torch.matmul(U2.weight.T, r2)
    r1 = r1 + U1(e1) 
    r2 = r2 + U2(e2)
    return e1, r1, e2, r2

# def step(x, e1, r1, e2, r2):
#     with torch.no_grad():
#         e1 = torch.matmul(U1.weight.T, r1) - x
#     r1 = r1 - U1(e1) 
#     with torch.no_grad():
#         e2 = torch.matmul(U2.weight.T, r2) - r1
#     r2 = r2 - U2(e2)
#     return e1, r1, e2, r2

In [311]:
x = torch.rand(2)
e1 = torch.rand(2)
r1 = torch.rand(4)
e2 = torch.rand(4)
r2 = torch.rand(6)

# print all
print(f"x: {x}")
print(f"e1: {e1}")
print(f"r1: {r1}")
print(f"e2: {e2}")
print(f"r2: {r2}")


x: tensor([0.8860, 0.5832])
e1: tensor([0.3376, 0.8090])
r1: tensor([0.5779, 0.9040, 0.5547, 0.3423])
e2: tensor([0.6343, 0.3644, 0.7104, 0.9464])
r2: tensor([0.7890, 0.2814, 0.7886, 0.5895, 0.7539, 0.1952])


In [322]:
e1 = e1.detach()
r1 = r1.detach()
e2 = e2.detach()
r2 = r2.detach()
e1, r1, e2, r2 = step(x, e1, r1, e2, r2)
print(f" x: {x}")
print(f"e1: {e1}")
print(f"r1: {r1}")
print(f"e2: {e2}")
print(f"r2: {r2}")

 x: tensor([0.8860, 0.5832])
e1: tensor([ 0.0021, -0.0002])
r1: tensor([ 1.2948,  0.1165,  0.1364, -0.6544], grad_fn=<AddBackward0>)
e2: tensor([ 0.1123,  0.0203, -0.1476, -0.1888])
r2: tensor([ 0.0125,  1.4115,  1.3945, -1.2080,  0.3884,  0.1340],
       grad_fn=<AddBackward0>)


In [197]:
with torch.no_grad():
    U1.weight.grad = torch.zeros_like(U1.weight)
    U2.weight.grad = torch.zeros_like(U2.weight)
loss = 0.5*((r1**2).sum() + (r2**2).sum())
loss.sum().backward()
print(f'U1.weight.grad:\n{U1.weight.grad}')
print(f'U2.weight.grad:\n{U2.weight.grad}')

U1.weight.grad:
tensor([[ 0.6933, -0.4221],
        [ 0.3558, -0.2166],
        [ 0.3064, -0.1865],
        [-0.1959,  0.1192]])
U2.weight.grad:
tensor([[ 0.3013,  0.6518,  0.1750, -0.1821],
        [ 0.2382,  0.5154,  0.1384, -0.1440],
        [ 0.3859,  0.8350,  0.2242, -0.2333],
        [ 0.1283,  0.2775,  0.0745, -0.0776],
        [ 0.1763,  0.3815,  0.1024, -0.1066],
        [-0.0057, -0.0124, -0.0033,  0.0035]])


In [198]:
print(f'1:\n{torch.matmul(e1.unsqueeze(1), r1.unsqueeze(1).T).T}')
print(f'2:\n{torch.matmul(e2.unsqueeze(1), r2.unsqueeze(1).T).T}')

1:
tensor([[-0.6933,  0.4221],
        [-0.3558,  0.2166],
        [-0.3064,  0.1865],
        [ 0.1959, -0.1192]], grad_fn=<PermuteBackward0>)
2:
tensor([[-0.3013, -0.6518, -0.1750,  0.1821],
        [-0.2382, -0.5154, -0.1384,  0.1440],
        [-0.3859, -0.8350, -0.2242,  0.2333],
        [-0.1283, -0.2775, -0.0745,  0.0776],
        [-0.1763, -0.3815, -0.1024,  0.1066],
        [ 0.0057,  0.0124,  0.0033, -0.0035]], grad_fn=<PermuteBackward0>)


In [274]:
U = nn.Conv2d(1, 3, (3,3), bias=False)
optimiser = torch.optim.SGD(U.parameters(), lr=0.001)
Ut = nn.ConvTranspose2d(3, 1, (3,3))
with torch.no_grad():
    Ut.weight = U.weight
grad = torch.zeros_like(U.weight)
def step(x, e, r):
    with torch.no_grad():
        e = x - Ut(r)
    r = r + U(e)
    return e, r

In [275]:
x = torch.rand((1,1,4,4))
e = torch.zeros_like(x)
r = torch.zeros((1,3,2,2))

In [294]:
e = e.detach()
r = r.detach()
e, r = step(x, e, r)
print(e)
print(r)

tensor([[[[ 0.5686, -0.0906,  0.0353,  0.2347],
          [ 0.3378,  0.1243, -0.1070, -0.0324],
          [ 0.2056,  0.1212, -0.0081, -0.0222],
          [ 0.0375,  0.0403,  0.0899,  0.0199]]]])
tensor([[[[ 0.3041,  0.3235],
          [-0.1773, -0.0444]],

         [[-0.0907, -1.0017],
          [ 0.0183, -0.6439]],

         [[ 0.6487,  1.1892],
          [-0.4736, -0.6325]]]], grad_fn=<AddBackward0>)


In [295]:
optimiser.zero_grad()
loss = (0.5*(r**2)).sum()
print(loss)
loss.backward()
optimiser.step()

tensor(2.0582, grad_fn=<SumBackward0>)


In [296]:
U.weight.grad

tensor([[[[ 0.0782, -0.0334,  0.1071],
          [ 0.1011, -0.0180, -0.0406],
          [ 0.0933,  0.0231, -0.0265]]],


        [[[-0.0347,  0.0440, -0.2194],
          [-0.2294,  0.1033,  0.0563],
          [-0.1653, -0.0601,  0.0118]]],


        [[[ 0.0226, -0.0079,  0.3732],
          [ 0.1929, -0.0989, -0.0901],
          [ 0.2343, -0.0069, -0.0868]]]])